# 1. Instruções

1. O projeto esteja conforme a opção de Área de Atuação selecionada na Plataforma de Processos Seletivos no momento da inscrição, caso contrário, o Projeto do candidato não será avaliado e será atribuída nota 0 (zero) à avaliação.
2. A entrega do Projeto seja realizada pela Plataforma de Processos Seletivos do LAIS.
3. A entrega do Projeto seja realizada até a data limite, conforme o cronograma do Edital e suas retificações.
4. O projeto esteja versionado em repositório público (GitHub/GitLab/BitBucket).
5. Seja enviado um arquivo PDF, contendo o link para o commit que será avaliado.
     1. Exemplo: https://github.com/lais-huol/capacitacaogit/commit/dc125ec4d914788f43d6b98114701f018338bc05.
     2. Não serão aceitos links fora do padrão exemplificado acima

A candidatura será eliminada se o candidato for avaliado com nota inferior a 35 (trinta e cinco) pontos na avaliação, obtiver nota inferior à mínima em cada critério e 
avaliação do "Quadro 2" ou não participar da entrevista no horário e canal definidos.

# 2. Descrição do Projeto
O Projeto visa construir um repositório de dados com informações de estabelecimentos de saúde do Brasil a partir da identificação dos atributos presentes, com base na fonte a seguir: 
 - https://drive.google.com/file/d/1B02BpVfhx_qMsuuJEnkcNtEUqCNU0dHJ/view?usp=sharing


Os dados deverão ser consumidos dos arquivos CSVs contidos no arquivo compactado acima, processados e armazenados em banco de dados NoSQL. O repositório deverá disponibilizar dados que serão utilizados para buscas textuais e consultas de agregação.

# 3. Requisitos 
1. Os dados devem ser armazenados em um banco de dados NoSQL, sendo MongoDB ou ElasticSearch.
2. O processo de extração, transformação e carregamento dos arquivos CSVs devem ser implementados por meio da linguagem de programação Python.
3. A plotagem dos dados deve ser implementada na linguagem Python e fazendo uso do Jupyter Notebook.

# 4. Requisitos para a apresentação de defesa
A defesa do projeto deve apresentar todo o rigor técnico necessário para a sua explanação, com objetividade, mas sem omitir informações que são importantes para a correta compreensão da proposta. Abaixo, estão listados os principais pontos que devem ser abordados pelo candidato: 
1. Descrever sucintamente o processo adotado no desenvolvimento do projeto.
2. Evidenciar que cumpriu os requisitos transversais.
3. Evidenciar ter disponibilidade para desempenhar as atribuições da bolsa, 
considerando suas outras atividades extrabolsa.
4. Evidenciar como sua trajetória acadêmica e profissional o qualificam para 
contribuir com o projeto objetivo da bolsa.
5. Ser pontual e ter o ambiente preparado para a entrevista

5. Critérios de Avaliação
A seguir estão listados os itens que serão avaliados na proposta, conforme requisitos do Projeto e seus respectivos pontos.

|**Entrevista com o candidato - Itens de avaliação (Total de 20 pontos)**| Nota|
|---|---|
|1. Demonstração de disponibilidade para desempenhar as atribuições decorrentes da bolsa.|20|
|2. Capacidade de explicitação da relação entre a vaga pretendida e sua trajetória formativa e profissional (considerando as áreas de conhecimentos elencados no item 5.1.g).|20|
|3. Capacidade de expor de maneira clara, objetiva e consistente suas ideias relacionadas à defesa do Projeto. |20|
|**Defesa do projeto (Obrigatórios - Total de 30 pontos)**|
|4. Processar (extrair, transformar e carregar) os arquivos CSVs.|15,0|
|5. Modelagem do JSON.| 5,0|
|6. Escrever um script Python no Jupyter Notebook para consumir os dados de estabelecimento dado o código CNES.| 5,0|
|7. Escrever um script Python no Jupyter Notebook para efetuar uma consulta de agregação da quantidade de estabelecimentos de saúde por tipo de estabelecimento de determinado município.|5,0|
|**Defesa do projeto (Bônus - Total de 20 pontos)**|
|8. Plotagem para apresentação dos dados com, no mínimo, 5 visualizações.|20,0|
|**Pontuação máxima**|70|

In [52]:
! pip install missingno pymongo python-dateutil

In [190]:
# Bibliotecas
# Tabelas
import pandas as pd
# Explore data
import missingno as msno
# Vetores
import numpy as np
# Gráficos
import seaborn as sns
import matplotlib.pyplot as plt
# Carregamento de arquivos
from zipfile import ZipFile as zp
# Banco de Dados
import pymongo

# 0. Funções

In [ ]:
def listColumnsChar(df, colName):
    """ 
        Função para explorar dados em colunas
    """
    aux = df[colName]
    display(aux.head())
    mask = aux.dtypes == 'O'
    for col in aux.loc[:, mask].columns:
        print(f"{listComposition(col, ''.join([str(x) for x in aux[col].unique().tolist()]))} Ex:{aux.loc[0, col]}")

def listComposition(col, val):
    """ 
        Verifica se um dado campo possui caracteres numéricos, especiais e alfabéticos.
        retorna uma string com três elementos que podem ser zero ou um, 1 quanto existe
        um elemento de um dado tipo e 0 caso contrário.
    """
    aux = np.unique(list(val))
    isCharNumSpec = ['0', '0', '0']
    for ii in aux:
        if ii.isalpha():
            isCharNumSpec[0] = '1'
        elif ii.isnumeric():
            isCharNumSpec[1] = '1'
        else:
            isCharNumSpec[2] = '1'

    return f'{col}: Char-Num-Spec: {"-".join(isCharNumSpec)} Charcters: {aux}'

def subsMonth(val, mDict):
    """
        Substitui o mês por seu equivalente numérico e retrona uma string
        ------
        Parameters:
        val: list
        mDict: dictionary
    """
    val[1] = mDict[val[1]]
    return '-'.join(val)

def fixDateString(serie, mDict):
    """
        Altera a string da coluna, mantendo apenas a data e subistituindo o mês por
        seu equivalente numérico, além de trocar os dados faltantes por 00-00-0000
    """
    serie = serie.apply(lambda x: x.split(' ', -1)[0] if type(x) != float else np.NAN)
    serie = serie.apply(lambda x: subsMonth(x.split('-'), mDict) if type(x) != float else np.NAN)
    return serie

# 1. Carregar arquivos

In [161]:
# Caminho do arquivo
filePath = 'D:\Ivan\OneDrive\Códigos ( Profissional )\LAIS\Personal_Proj_Lais_Data_Engineer\Data\ARQUIVOS_EDITAL_43_2021_PERFIL_X_DATA_ENGINEERING.zip'

sourceFile = zp(filePath)                                           # Carrega o arquivo
filesInSource = sourceFile.namelist()                               # Lista arquivos dentro
rawDataDict = {k.split('/', -1)[-1].split('.')[0]: pd.read_csv(sourceFile.open(k), sep=';', na_values=['undefined'], low_memory=False) for k in filesInSource} # Carrega cada tabela e armazena num dicionário
dctKeys = [*rawDataDict]
dctKeys

['tbEstabelecimento202107',
 'tbEstado202107',
 'tbMotivoDesativacao202107',
 'tbMunicipio202107',
 'tbNaturezaJuridica202107',
 'tbTipoEstabelecimento202107',
 'tbTipoUnidade202107']

# 2. Processamento

In [186]:
# Eplorar dados
listColumnsChar(rawDataDict, dctKeys[1])

,CO_UF,CO_SIGLA,NO_DESCRICAO
0,27,AL,ALAGOAS
1,13,AM,AMAZONAS
2,29,BA,BAHIA
3,23,CE,CEARA
4,50,MS,MATO GROSSO DO SUL


CO_SIGLA: Char-Num-Spec: 1-0-0 Charcters: ['A' 'B' 'C' 'D' 'E' 'F' 'G' 'I' 'J' 'L' 'M' 'N' 'O' 'P' 'R' 'S' 'T'] Ex:AL
NO_DESCRICAO: Char-Num-Spec: 1-0-1 Charcters: [' ' 'A' 'B' 'C' 'D' 'E' 'F' 'G' 'H' 'I' 'J' 'L' 'M' 'N' 'O' 'P' 'R' 'S'
 'T' 'U' 'Z'] Ex:ALAGOAS


In [162]:
table = rawDataDict[dctKeys[0]]
dtMask = table.columns.str.contains("DT")
# Dados antes da alteração
display(table.loc[:, dtMask].head())

# Correção de datetime
monthDict = {'jan': '01', 'fev': '02', 'mar': '03', 'abr': '04', 'mai': '05', 'jun': '06', 'jul': '07', 'ago': '08', 'set':'09', 'out':'10', 'nov': '11', 'dez': '12', '00': '00'}

cols = ['DT_EXPEDICAO','DT_VAL_LIC_SANI']
for ii in cols:
    table[ii] = fixDateString(table[ii], monthDict)
    table[ii] = pd.to_datetime(table[ii], format='%d-%m-%Y', errors='ignore')

cols = ["TO_CHAR(DT_ATUALIZACAO,'DD/MM/YYYY')", "TO_CHAR(DT_ATU_GEO,'DD/MM/YYYY')",	"TO_CHAR(DT_ATUALIZACAO_ORIGEM,'DD/MM/YYYY')"]
for ii in cols:
    table[ii] = pd.to_datetime(table[ii], format='%d/%m/%Y', errors='ignore')
    
# Padronização de Latitude e Longitude: ',' -> '.'
cols = ['NU_LATITUDE', 'NU_LONGITUDE']
for ii in cols:
    table[ii].apply(lambda x: x.replace(',', '.')  if type(x) != float else np.NAN)

# Resultado
table.loc[:, ['DT_EXPEDICAO','DT_VAL_LIC_SANI',"TO_CHAR(DT_ATUALIZACAO,'DD/MM/YYYY')", "TO_CHAR(DT_ATU_GEO,'DD/MM/YYYY')",	"TO_CHAR(DT_ATUALIZACAO_ORIGEM,'DD/MM/YYYY')",'NU_LATITUDE', 'NU_LONGITUDE']].head()

,DT_EXPEDICAO,DT_VAL_LIC_SANI,"TO_CHAR(DT_ATUALIZACAO,'DD/MM/YYYY')","TO_CHAR(DT_ATU_GEO,'DD/MM/YYYY')","TO_CHAR(DT_ATUALIZACAO_ORIGEM,'DD/MM/YYYY')"
0,30-jun-2021 00:00:00,30-jun-2022 00:00:00,26/07/2021,26/07/2021,01/08/2021
1,NaN,NaN,28/07/2021,28/07/2021,01/08/2021
2,NaN,NaN,28/07/2021,28/07/2021,03/08/2021
3,21-jun-2021 00:00:00,30-abr-2022 00:00:00,05/07/2021,02/07/2021,06/07/2021
4,08-jul-2021 00:00:00,08-jul-2023 00:00:00,08/07/2021,08/07/2021,16/07/2021


,DT_EXPEDICAO,DT_VAL_LIC_SANI,"TO_CHAR(DT_ATUALIZACAO,'DD/MM/YYYY')","TO_CHAR(DT_ATU_GEO,'DD/MM/YYYY')","TO_CHAR(DT_ATUALIZACAO_ORIGEM,'DD/MM/YYYY')",NU_LATITUDE,NU_LONGITUDE
0,30-06-2021,30-06-2022,2021-07-26,2021-07-26,2021-08-01,-26.85368475,-49.120375519202895
1,NaN,NaN,2021-07-28,2021-07-28,2021-08-01,-26.9155558,-48.653778
2,NaN,NaN,2021-07-28,2021-07-28,2021-08-03,-9.975,-67.81
3,21-06-2021,30-04-2022,2021-07-05,2021-07-02,2021-07-06,-22.5117723,-44.086488
4,08-07-2021,08-07-2023,2021-07-08,2021-07-08,2021-07-16,-11.6671379,-61.1914104


# 3. Conexão MongoDB

In [41]:
connectionUrl = 'mongodb://localhost:27017/?readPreference=primary&appname=MongoDB%20Compass&directConnection=true&ssl=false'
client = pymongo.MongoClient(connectionUrl)

In [48]:
def get_database():
    from pymongo import MongoClient
    import pymongo

    # Provide the mongodb atlas url to connect python to mongodb using pymongo
    CONNECTION_STRING = "mongodb://localhost:27017/?readPreference=primary&appname=MongoDB%20Compass&directConnection=true&ssl=false"

    # Create a connection using MongoClient. You can import MongoClient or use pymongo.MongoClient
    from pymongo import MongoClient
    client = MongoClient(CONNECTION_STRING)

    # Create the database for our example (we will use the same database throughout the tutorial
    return client['user_shopping_list']
    
# This is added so that many files can reuse the function get_database()
if __name__ == "__main__":    
    
    # Get the database
    dbname = get_database()

In [50]:
collection_name = dbname["user_1_items"]

In [51]:
item_1 = {
"_id" : "U1IT00001",
"item_name" : "Blender",
"max_discount" : "10%",
"batch_number" : "RR450020FRG",
"price" : 340,
"category" : "kitchen appliance"
}

item_2 = {
"_id" : "U1IT00002",
"item_name" : "Egg",
"category" : "food",
"quantity" : 12,
"price" : 36,
"item_description" : "brown country eggs"
}
collection_name.insert_many([item_1,item_2])